#### Creating README.txt
# OBSERVATION: 
If you have doubts about the code, contact rdr-contacte@csuc.cat

## SCRIPT OBJECTIVE
The main objective of this script is to automatically create the README file for a dataset.


In [ ]:
# @title Install or Update Libraries (Click the &#x25B6; button to execute)

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Function to install required packages
def install_packages(b):
    """
    Function to install or update necessary Python packages.

    Parameters:
    - b: Button object. Click event handler.
    """
    clear_output(wait=True)
    !pip install --upgrade pip -q  # Upgrade pip silently
    !pip install pyDataverse -q  # Install or upgrade pyDataverse silently
    !pip install html2text -q  # Install or upgrade html2text silently
    print("Libraries have been downloaded or updated.")

# Displaying installation message
display(HTML("<p style='font-size:14px;'><b>Click the following button to install the libraries.</b></p>"))

# Creating installation button
install_button = widgets.Button(description='Install Libraries')
install_button.on_click(install_packages)

# Displaying the installation button
display(install_button)


In [ ]:
# @title Enter DOI (doi:10.34810/dataXXX), API token and the repository URL. Click the &#x25B6; button to execute cell

import os
from IPython.display import display, FileLink
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
from pyDataverse.api import NativeApi, DataAccessApi
from pyDataverse.models import Dataverse

# Provide input values
doi = ""  # @param {type:"string"}  
token = ""  # @param {type:"string"}  
base_url = "https://dataverse.csuc.cat/" # @param {type:"string"}

def extract_value(data_dict):
    """
    Function to extract all keys and values from a JSON metadata dictionary.

    Parameters:
    - data_dict: dict. JSON metadata dictionary.

    Returns:
    - type_names: list. List of type names extracted from the metadata.
    - values: list. List of values extracted from the metadata.
    """
    if isinstance(data_dict, dict):
        type_names = []
        values = []
        for key, value in data_dict.items():
            if key == 'typeName' and 'value' in data_dict:
                if isinstance(data_dict['value'], list):
                    for v in data_dict['value']:
                        type_names.append(data_dict['typeName'])
                        values.append(v)
                else:
                    type_names.append(data_dict['typeName'])
                    values.append(data_dict['value'])
            elif isinstance(value, dict) and 'typeName' in value and 'value' in value:
                type_names.append(value['typeName'])
                values.append(value['value'])
            elif isinstance(value, str) and key == 'typeName':
                type_names.append(value)
                values.append(value)
            else:
                extracted_type_names, extracted_values = extract_value(value)
                type_names += extracted_type_names
                values += extracted_values
        return type_names, values
    elif isinstance(data_dict, list):
        type_names = []
        values = []
        for item in data_dict:
            extracted_type_names, extracted_values = extract_value(item)
            type_names += extracted_type_names
            values += extracted_values
        return type_names, values
    else:
        return [], []
def exportmetadata(base_url, token, doi, citation_keys, citation_values, geo_keys, geo_values, social_keys, social_values, astronomy_keys, astronomy_values, biomedical_keys, biomedical_values, journal_keys, journal_values):
    """
    Function to export metadata from a dataset identified by its DOI.

    Parameters:
    - base_url: str. Base URL of the Dataverse instance.
    - token: str. API token for authentication.
    - doi: str. DOI of the dataset.
    - citation_keys: list. List to store citation metadata keys.
    - citation_values: list. List to store citation metadata values.
    - geo_keys: list. List to store geospatial metadata keys.
    - geo_values: list. List to store geospatial metadata values.
    - social_keys: list. List to store social science metadata keys.
    - social_values: list. List to store social science metadata values.
    - astronomy_keys: list. List to store astronomy metadata keys.
    - astronomy_values: list. List to store astronomy metadata values.
    - biomedical_keys: list. List to store biomedical metadata keys.
    - biomedical_values: list. List to store biomedical metadata values.
    - journal_keys: list. List to store journal metadata keys.
    - journal_values: list. List to store journal metadata values.

    Returns:
    - None. Updates the provided lists with extracted metadata.
    """
    from pyDataverse.api import NativeApi, DataAccessApi
    from pyDataverse.models import Dataverse
    import html2text
    import os

    # Instantiate API objects for accessing Dataverse
    api = NativeApi(base_url, token)
    data_api = DataAccessApi(base_url, token)

    try:
        # Retrieve dataset metadata
        dataset = api.get_dataset(doi)

        # Extract citation metadata if available
        if 'citation' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_citation = dataset.json()['data']['latestVersion']['metadataBlocks']['citation']['fields']
            citation = extract_value(metadata_citation)
            citation_keys.extend(citation[0])
            citation_values.extend(citation[1])
            for item in metadata_citation:
                if isinstance(item['value'], str):
                    index_canvi = citation_keys.index(item['typeName'])
                    citation_values[index_canvi] = item['value']

        # Extract geospatial metadata if available
        if 'geospatial' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_geospatial = dataset.json()['data']['latestVersion']['metadataBlocks']['geospatial']['fields']
            geospatial = extract_value(metadata_geospatial)
            geo_keys.extend(geospatial[0])
            geo_values.extend(geospatial[1])
            for item in metadata_geospatial:
                if isinstance(item['value'], str):
                    index_canvi = geo_keys.index(item['typeName'])
                    geo_values[index_canvi] = item['value']

        # Extract social science metadata if available
        if 'socialscience' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_socialscience = dataset.json()['data']['latestVersion']['metadataBlocks']['socialscience']['fields']
            socialscience = extract_value(metadata_socialscience)
            social_keys.extend(socialscience[0])
            social_values.extend(socialscience[1])
            for item in metadata_socialscience:
                if isinstance(item['value'], str):
                    index_canvi = social_keys.index(item['typeName'])
                    social_values[index_canvi] = item['value']

        # Extract astronomy metadata if available
        if 'astrophysics' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_astronomy = dataset.json()['data']['latestVersion']['metadataBlocks']['astrophysics']['fields']
            astronomy = extract_value(metadata_astronomy)
            astronomy_keys.extend(astronomy[0])
            astronomy_values.extend(astronomy[1])
            for item in metadata_astronomy:
                if isinstance(item['value'], str):
                    index_canvi = astronomy_keys.index(item['typeName'])
                    astronomy_values[index_canvi] = item['value']

        # Extract biomedical metadata if available
        if 'biomedical' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_biomedical = dataset.json()['data']['latestVersion']['metadataBlocks']['biomedical']['fields']
            biomedical = extract_value(metadata_biomedical)
            biomedical_keys.extend(biomedical[0])
            biomedical_values.extend(biomedical[1])
            for item in metadata_biomedical:
                if isinstance(item['value'], str):
                    index_canvi = biomedical_keys.index(item['typeName'])
                    biomedical_values[index_canvi] = item['value']

        # Extract journal metadata if available
        if 'journal' in dataset.json()['data']['latestVersion']['metadataBlocks']:
            metadata_journal = dataset.json()['data']['latestVersion']['metadataBlocks']['journal']['fields']
            journal = extract_value(metadata_journal)
            journal_keys.extend(journal[0])
            journal_values.extend(journal[1])
            for item in metadata_journal:
                if isinstance(item['value'], str):
                    index_canvi = journal_keys.index(item['typeName'])
                    journal_values[index_canvi] = item['value']

    except (KeyError, InvalidSchema) as e:
        # Catch specific exceptions and print the error message
        print(f"Error occurred: {e}")
        print('There was an error reading metadata for the dataset: ' + doi)

def filemetadata(base_url, token, doi, filemetadata_keys, filemetadata_values):
    """
    Function to extract metadata for files associated with a dataset identified by its DOI.

    Parameters:
    - base_url: str. Base URL of the Dataverse instance.
    - token: str. API token for authentication.
    - doi: str. DOI of the dataset.
    - filemetadata_keys: list. List to store file metadata keys.
    - filemetadata_values: list. List to store file metadata values.

    Returns:
    - None. Updates the provided lists with extracted file metadata.
    """
    from pyDataverse.api import NativeApi, DataAccessApi
    from pyDataverse.models import Dataverse

    # Instantiate API objects for accessing Dataverse
    api = NativeApi(base_url, token)
    data_api = DataAccessApi(base_url, token)

    try:
        # Retrieve dataset metadata
        dataset = api.get_dataset(doi)

        # Iterate through files and extract metadata
        for i in range(len(dataset.json()['data']['latestVersion']['files'])):
            filemetadata_resp = dataset.json()['data']['latestVersion']['files'][i]['dataFile']
            filemetadata_keys_aux = list(filemetadata_resp.keys())
            filemetadata_values_aux = list(filemetadata_resp.values())
            filemetadata_keys.append(filemetadata_keys_aux)
            filemetadata_values.append(filemetadata_values_aux)
    except KeyError:
        print('There was an error reading metadata for the files of the dataset: ' + doi)

def list_duplicates_of(seq, item):
    """
    Function to list indexes of duplicates of an item in a sequence.

    Parameters:
    - seq: list. Sequence to search for duplicates.
    - item: any. Item to search for duplicates.

    Returns:
    - locs: list. List of indexes where the item occurs more than once in the sequence.
    """
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item, start_at + 1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

def find_keys(keys, specified_keys, values):
    """
    Function to find specified keys and their corresponding values in a list of keys and values.

    Parameters:
    - keys: list. List of keys.
    - specified_keys: list. List of specified keys to find.
    - values: list. List of values corresponding to the keys.

    Returns:
    - extracted_values: list. List of dictionaries containing extracted key-value pairs.
    """
    # Dictionary to store extracted values
    extracted_values = []
    # Dictionary to store current entry
    current_entry = {}
    # Set to keep track of found keys
    found_keys = set()
    # Iterate through keys and values
    for key, value in zip(keys, values):
        # Check if current key is in specified keys
        if key in specified_keys:
            current_entry[key] = value
            found_keys.add(key)
            # If all specified keys are found, add entry to extracted_values
            if len(found_keys) == len(specified_keys):
                extracted_values.append(current_entry)
                current_entry = {}  # Reset current entry
                found_keys.clear()  # Clear found keys set for next entry
    return extracted_values

def format_key(key):
    """
    Function to format a key by splitting camel case and capitalizing the first letter of each word.

    Parameters:
    - key: str. Key to be formatted.

    Returns:
    - formatted_key: str. Formatted key.
    """
    words = []
    current_word = ''
    for char in key:
        if char.isupper() and current_word:
            words.append(current_word)
            current_word = char
        else:
            current_word += char
    if current_word:
        words.append(current_word)

    formatted_key = ' '.join(words)
    return formatted_key.capitalize()

def createreadme(base_url, token, doi, citation_keys, citation_values, geo_keys, geo_values, social_keys, social_values, astronomy_keys, astronomy_values, biomedical_keys, biomedical_values, journal_keys, journal_values, filemetadata_keys, filemetadata_values):
    """
    Function to create a readme file for a dataset.

    Parameters:
    - base_url: str. Base URL of the Dataverse instance.
    - token: str. API token for authentication.
    - doi: str. DOI of the dataset.
    - citation_keys: list. List of citation metadata keys.
    - citation_values: list. List of citation metadata values.
    - geo_keys: list. List of geospatial metadata keys.
    - geo_values: list. List of geospatial metadata values.
    - social_keys: list. List of social science metadata keys.
    - social_values: list. List of social science metadata values.
    - astronomy_keys: list. List of astronomy metadata keys.
    - astronomy_values: list. List of astronomy metadata values.
    - biomedical_keys: list. List of biomedical metadata keys.
    - biomedical_values: list. List of biomedical metadata values.
    - journal_keys: list. List of journal metadata keys.
    - journal_values: list. List of journal metadata values.
    - filemetadata_keys: list. List of file metadata keys.
    - filemetadata_values: list. List of file metadata values.
    """

    # Import necessary libraries
    from pyDataverse.api import NativeApi, DataAccessApi
    from pyDataverse.models import Dataverse
    import os
    import html2text

    # Instantiate API objects for accessing Dataverse
    api = NativeApi(base_url, token)
    data_api = DataAccessApi(base_url, token)

    # Retrieve dataset metadata
    dataset = api.get_dataset(doi)

    # Extract path from DOI
    path = doi.replace("doi:10.34810/", "")

    try:
        # Create directory if it does not exist
        os.mkdir(path)
    except OSError:
        print("Directory " + path + ' already exists. The Readme will be saved in this directory.')

    with open(path + '/' + 'Readme.txt', 'w', encoding='utf-8') as f:
        f.write('GENERAL INFORMATION\n------------------\n')
        cont = 0

        # Write metadata to Readme file
        if 'PreviousDatasetPersistentID' in citation_keys:
            cont += 1
            f.write(str(cont) + '.  Previous Dataset Persistent ID:\n')
            auxiliar = []
            auxiliar.append(list_duplicates_of(citation_keys, 'PreviousDatasetPersistentID'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')

        if 'title' in citation_keys:
            cont += 1
            f.write(str(cont) + '.  Dataset title:\n')
            f.write('\t' + citation_values[citation_keys.index('title')] + '\n\n')  # Write the title

        if 'authorName' in citation_keys or 'authorAffiliation' in citation_keys or 'authorIdentifierScheme' in citation_keys or 'authorIdentifier' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Authorship:\n')
            keys=['authorName','authorAffiliation','authorIdentifierScheme','authorIdentifier' ]
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'datasetContactName' in citation_keys or 'datasetContactAffiliation' in citation_keys or 'datasetContactEmail' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Dataset contact:\n')
            keys=['datasetContactName','datasetContactAffiliation','datasetContactEmail' ]
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        f.write('DESCRIPTION\n----------\n')
        cont=0
        if 'language' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Dataset language:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'language'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'dsDescriptionValue' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Abstract:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'dsDescriptionValue'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'subject' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Subject:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'subject'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'keywordValue' in citation_keys or 'keywordVocabulary' in citation_keys or 'keywordVocabularyURI' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Keyword:\n')
            keys=['keywordValue','keywordVocabulary','keywordVocabularyURI' ]
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'topicClassValue' in citation_keys or 'topicClassVocab' in citation_keys or 'topicClassVocabURI' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Topic classification:\n')
            keys=['topicClassValue','topicClassVocab','topicClassVocabURI' ]
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'notesText' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Notes:\n')
            f.write('\t'+citation_values[citation_keys.index('notesText')]+'\n\n')
        if 'producerName' in citation_keys or 'producerAffiliation' in citation_keys or 'producerAbbreviation' in citation_keys or 'producerURL' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Producer:\n')
            keys=['producerName','producerAffiliation','producerAbbreviation','producerURL' ]
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'productionDate' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Production date:\n')
            f.write('\t'+citation_values[citation_keys.index('productionDate')]+'\n\n')
        if 'Production place' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Production place:\n')
            f.write('\t'+citation_values[citation_keys.index('productionPlace')]+'\n\n')
        if 'contributorType' in citation_keys or 'contributorName' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Contributor:\n')
            keys=['contributorType','contributorName']
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'grantNumberAgency' in citation_keys or 'grantNumberValue' in citation_keys:
            cont+=1
            f.write(str(cont)+'. Grant information:\n')
            keys=['grantNumberAgency','grantNumberValue']
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'distributorName' in citation_keys or 'distributorAffiliation' in citation_keys or 'distributorAbbreviation' in citation_keys or 'distributorURL' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Distributor:\n')
            keys=['distributorName','distributorAffiliation','distributorAbbreviation','distributorURL' ]
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'distributionDate' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Distribution date:\n')
            f.write('\t'+citation_values[citation_keys.index('distributionDate')]+'\n\n')
        if 'depositor' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Depositor:\n')
            f.write('\t'+citation_values[citation_keys.index('depositor')]+'\n\n')
        if 'dateOfDeposit' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Date of deposit:\n')
            f.write('\t'+citation_values[citation_keys.index('dateOfDeposit')]+'\n\n')
        if 'timePeriodCoveredStart' in citation_keys or 'timePeriodCoveredEnd' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Time period covered (single date or date range):\n')
            keys=['timePeriodCoveredStart','timePeriodCoveredEnd']
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'dateOfCollectionStart' in citation_keys or 'dateOfCollectionEnd' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Date of data collection (single date or date range):\n')
            keys=['dateOfCollectionStart','dateOfCollectionEnd']
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'publicationDate' in dataset.json()['data']:
            cont+=1
            f.write(str(cont)+'.  Publication Date:\n')
            f.write('\t'+dataset.json()['data']['publicationDate']+'\n\n')
        if 'dateOfCollectionStart' not in citation_keys and 'publicationDate' not in dataset.json()['data']:
            f.write('\n')
        if 'kindOfData' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Kind of Data:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'kindOfData'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'seriesName' in citation_keys or 'seriesInformation' in citation_keys:
            cont+=1
            f.write(str(cont)+'. Series:\n')
            index=[]
            keys=['seriesName','seriesInformation']
            specified_keys=[]
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'softwareName' in citation_keys or 'softwareVersion' in citation_keys:
            cont+=1
            f.write(str(cont)+'. Software:\n')
            keys=['softwareName','softwareVersion']
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if 'relatedMaterial' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Related material:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'relatedMaterial'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'relatedDatasets' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Related datasets:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'relatedDatasets'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'otherReferences' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Other references:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'otherReferences'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'dataSources' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Data sources:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'dataSources'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'originOfSources' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Origin of sources:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'originOfSources'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'characteristicOfSources' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Characteristic of sources:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'characteristicOfSources'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        if 'accessToSources' in citation_keys:
            cont+=1
            f.write(str(cont)+'.  Access to sources:\n')
            auxiliar=[]
            auxiliar.append(list_duplicates_of(citation_keys, 'accessToSources'))
            for i in auxiliar[0]:
                f.write('\t')
                f.write(citation_values[i])
                if i != auxiliar[0][-1]:
                    f.write('\n ')
            f.write('\n\n')
        f.write('ACCESS INFORMATION\n------------------------\n')
        cont=0
        if 'license' in dataset.json()['data']['latestVersion']:
            cont+=1
            f.write(str(cont)+'. Creative Commons License of the dataset: \n')
            f.write('\t'+dataset.json()['data']['latestVersion']['license']['name']+'\n\n')
        if 'persistentUrl' in dataset.json()['data']:
            cont+=1
            f.write(str(cont)+'. Dataset DOI: \n')
            f.write('\t'+dataset.json()['data']['persistentUrl']+'\n\n')
        if 'publicationCitation' in citation_keys or 'publicationIDType'in citation_keys or 'publicationIDNumber' in citation_keys or 'publicationURL' in citation_keys:
            cont+=1
            f.write(str(cont)+'. Related publication:\n')
            index=[]
            keys=['publicationCitation','publicationIDType','publicationIDNumber','publicationURL']
            specified_keys = [element for element in keys if element in citation_keys]
            extracted_values = find_keys(citation_keys, specified_keys, citation_values)
            for entry in extracted_values:
                for key, value in entry.items():
                    formatted_key = format_key(key)
                    f.write('\t'+f'{formatted_key}: {value}\n')
                f.write('\n')
        if len(geo_keys) != 0:
            f.write('Geospatial Metadata \n--------------------------------------\n')
            cont=0
            if 'country' in geo_keys or 'state' in geo_keys or 'city' in geo_keys or 'otherGeographicCoverage' in geo_keys:
                cont+=1
                f.write(str(cont)+'. Geographical location/s of data collection:\n')
                keys=['country','state','city','otherGeographicCoverage']
                specified_keys=[]
                specified_keys = [element for element in keys if element in geo_keys]
                extracted_values = find_keys(geo_keys, specified_keys, geo_values)
                for entry in extracted_values:
                    for key, value in entry.items():
                        formatted_key = format_key(key)
                        f.write('\t'+f'{formatted_key}: {value}\n')
                    f.write('\n')
            if 'geographicUnit' in geo_keys:
                cont+=1
                f.write(str(cont)+'.  Geographic Unit:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(geo_keys, 'geographicUnit'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(geo_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'westLongitude' in geo_keys or 'eastLongitude' in geo_keys or 'northLongitude' in geo_keys or 'southLongitude' in geo_keys:
                keys=['westLongitude','eastLongitude','northLongitude','southLongitude']
                specified_keys=[]
                specified_keys = [element for element in keys if element in geo_keys]
                extracted_values = find_keys(geo_keys, specified_keys, geo_values)
                for entry in extracted_values:
                    for key, value in entry.items():
                        formatted_key = format_key(key)
                        f.write('\t'+f'{formatted_key}: {value}\n')
                    f.write('\n')
        if len(social_keys) != 0:
            f.write('Social Science and Humanities Metadata \n--------------------------------------\n')
            cont=0
            if 'unitOfAnalysis' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Unit of Analysis:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(social_keys, 'unitOfAnalysis'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(social_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'universe' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Universe:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(social_keys, 'universe'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(social_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'timeMethod' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Time method:\n')
                f.write('\t'+social_values[social_keys.index('timeMethod')]+'\n\n')
            if 'dataCollector' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Data Collector:\n')
                f.write('\t'+social_values[social_keys.index('dataCollector')]+'\n\n')
            if 'collectorTraining' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Collector Training:\n')
                f.write('\t'+social_values[social_keys.index('collectorTraining')]+'\n\n')
            if 'frequencyOfDataCollection' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Frequency:\n')
                f.write('\t'+social_values[social_keys.index('frequencyOfDataCollection')]+'\n\n')
            if 'samplingProcedure' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Sampling Procedure:\n')
                f.write('\t'+social_values[social_keys.index('samplingProcedure')]+'\n\n')
            if 'targetSampleActualSize' in social_keys or 'targetSampleSizeFormula' in social_keys:
                cont+=1
                f.write(str(cont)+'. Target Sample Size:\n')
                index=[]
                keys=['targetSampleActualSize','targetSampleSizeFormula']
                specified_keys=[]
                specified_keys = [element for element in keys if element in social_keys]
                extracted_values = find_keys(social_keys, specified_keys, social_values)
                for entry in extracted_values:
                    for key, value in entry.items():
                        formatted_key = format_key(key)
                        f.write('\t'+f'{formatted_key}: {value}\n')
                    f.write('\n')
            if 'deviationsFromSampleDesign' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Major Deviations for Sample Design:\n')
                f.write('\t'+social_values[social_keys.index('deviationsFromSampleDesign')]+'\n\n')
            if 'collectionMode' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Collection Mode:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(social_keys, 'collectionMode'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(social_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'researchInstrument' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Type of Research Instrument:\n')
                f.write('\t'+social_values[social_keys.index('researchInstrument')]+'\n\n')
            if 'dataCollectionSituation' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Characteristics of Data Collection Situation:\n')
                f.write('\t'+social_values[social_keys.index('dataCollectionSituation')]+'\n\n')
            if 'actionsToMinimizeLoss' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Actions to Minimize Losses:\n')
                f.write('\t'+social_values[social_keys.index('actionsToMinimizeLoss')]+'\n\n')
            if 'controlOperations' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Control Operations:\n')
                f.write('\t'+social_values[social_keys.index('controlOperations')]+'\n\n')
            if 'weighting' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Weighting:\n')
                f.write('\t'+social_values[social_keys.index('weighting')]+'\n\n')
            if 'cleaningOperations' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Cleaning Operations:\n')
                f.write('\t'+social_values[social_keys.index('cleaningOperations')]+'\n\n')
            if 'datasetLevelErrorNotes' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Study Level Error Notes:\n')
                f.write('\t'+social_values[social_keys.index('datasetLevelErrorNotes')]+'\n\n')
            if 'responseRate' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Response Rate:\n')
                f.write('\t'+social_values[social_keys.index('responseRate')]+'\n\n')
            if 'samplingErrorEstimates' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Estimates of Sampling Error:\n')
                f.write('\t'+social_values[social_keys.index('samplingErrorEstimates')]+'\n\n')
            if 'otherDataAppraisal' in social_keys:
                cont+=1
                f.write(str(cont)+'.  Other Forms of Data Appraisal:\n')
                f.write('\t'+social_values[social_keys.index('otherDataAppraisal')]+'\n\n')
            if 'socialScienceNotesType' in social_keys or 'socialScienceNotesSubject' in social_keys or 'socialScienceNotesText' in social_keys :
                cont+=1
                f.write(str(cont)+'. Notes:\n')
                index=[]
                keys=['socialScienceNotesType','socialScienceNotesSubject','socialScienceNotesText']
                specified_keys=[]
                specified_keys = [element for element in keys if element in social_keys]
                extracted_values = find_keys(social_keys, specified_keys, social_values)
                for entry in extracted_values:
                    for key, value in entry.items():
                        formatted_key = format_key(key)
                        f.write('\t'+f'{formatted_key}: {value}\n')
                    f.write('\n')
        if len(astronomy_keys) != 0:
            f.write('Astronomy and Astrophysics Metadata  \n--------------------------------------\n')
            cont=0
            if 'astroType' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Type:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(astronomy_keys, 'astroType'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(astronomy_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'astroFacility' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Facility:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(astronomy_keys, 'astroFacility'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(astronomy_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'astroInstrument' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Instrument:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(astronomy_keys, 'astroInstrument'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(astronomy_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'astroObject' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Object:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(astronomy_keys, 'astroObject'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(astronomy_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'resolution.Spatial' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Spatial Resolution:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('resolution.Spatial')]+'\n\n')
            if 'resolution.Spectral' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Spectral Resolution:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('resolution.Spectral')]+'\n\n')
            if 'resolution.Temporal' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Temporal Resolution:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('resolution.Temporal')]+'\n\n')
            if 'coverage.Spectral.Bandpass' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Bandpass:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(astronomy_keys, 'coverage.Spectral.Bandpass'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(astronomy_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'coverage.Spectral.CentralWavelength' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Central Wavelength (m):\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(astronomy_keys, 'coverage.Spectral.CentralWavelength'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(astronomy_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'coverage.Spectral.MinimumWavelength' in astronomy_keys or 'coverage.Spectral.MaximumWavelength' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'. Wavelength Range:\n')
                index=[]
                keys=['coverage.Spectral.MinimumWavelength','coverage.Spectral.MaximumWavelength']
                specified_keys=[]
                specified_keys = [element for element in keys if element in astronomy_keys]
                extracted_values = find_keys(astronomy_keys, specified_keys, astronomy_values)
                for entry in extracted_values:
                    for key, value in entry.items():
                        formatted_key = format_key(key)
                        f.write('\t'+f'{formatted_key}: {value}\n')
                    f.write('\n')
            if  'coverage.Temporal.StartTime' in astronomy_keys or  'coverage.Temporal.StartTime' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'. Dataset Date Range:\n')
                index=[]
                keys=['coverage.Temporal.StartTime', 'coverage.Temporal.StartTime']
                specified_keys=[]
                specified_keys = [element for element in keys if element in astronomy_keys]
                extracted_values = find_keys(astronomy_keys, specified_keys, astronomy_values)
                for entry in extracted_values:
                    for key, value in entry.items():
                        formatted_key = format_key(key)
                        f.write('\t'+f'{formatted_key}: {value}\n')
                    f.write('\n')
            if 'coverage.Spatial' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Sky Coverage:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(astronomy_keys, 'coverage.Spatial'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(astronomy_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'coverage.Depth' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Depth Coverage:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('coverage.Depth')]+'\n\n')
            if 'coverage.ObjectDensity' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Object Density:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('coverage.ObjectDensity')]+'\n\n')
            if 'coverage.ObjectCount' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Object Count:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('coverage.ObjectCount')]+'\n\n')
            if 'coverage.SkyFraction' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Fraction of Sky :\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('coverage.SkyFraction')]+'\n\n')
            if 'coverage.Polarization' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Polarization:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('coverage.Polarization')]+'\n\n')
            if 'redshiftType' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  RedshiftType:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('redshiftType')]+'\n\n')
            if 'resolution.Redshift' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'.  Redshift Resolution:\n')
                f.write('\t'+astronomy_values[astronomy_keys.index('resolution.Redshift')]+'\n\n')
            if  'coverage.Redshift.MinimumValue' in astronomy_keys or  'coverage.Redshift.MaximumValue' in astronomy_keys:
                cont+=1
                f.write(str(cont)+'. Redshift Value:\n')
                index=[]
                keys=['coverage.Redshift.MinimumValue','coverage.Redshift.MaximumValue']
                specified_keys=[]
                specified_keys = [element for element in keys if element in astronomy_keys]
                extracted_values = find_keys(astronomy_keys, specified_keys, astronomy_values)
                for entry in extracted_values:
                    for key, value in entry.items():
                        formatted_key = format_key(key)
                        f.write('\t'+f'{formatted_key}: {value}\n')
                    f.write('\n')
        if len(biomedical_keys) != 0:
            f.write('Life Sciences Metadata \n-------------------------\n')
            cont=0
            if 'studyDesignType' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Design Type:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyDesignType'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'studyFactorType' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Factor Type:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyFactorType'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'studyAssayOrganism' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Organism:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyAssayOrganism'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'studyAssayOtherOrganism' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Other Organism:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyAssayOtherOrganism'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'studyAssayMeasurementType' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Measurement Type:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyAssayMeasurementType'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'studyAssayOtherMeasurmentType' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Other Measurement Type:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyAssayOtherMeasurmentType'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'studyAssayTechnologyType' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Technology Type:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyAssayTechnologyType'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'studyAssayPlatform' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Technology Platform:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyAssayPlatform'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
            if 'studyAssayCellType' in biomedical_keys:
                cont+=1
                f.write(str(cont)+'.  Cell Type:\n')
                auxiliar=[]
                auxiliar.append(list_duplicates_of(biomedical_keys, 'studyAssayCellType'))
                for i in auxiliar[0]:
                    f.write('\t')
                    f.write(biomedical_values[i])
                    if i != auxiliar[0][-1]:
                        f.write('\n ')
                f.write('\n\n')
        if len(journal_keys) != 0:
            f.write('Journal Metadata \n-------------------------\n')
            cont=0
            if 'journalVolume' in journal_keys or 'journalIssue' in journal_keys or 'journalPubDate' in journal_keys:
                cont+=1
                f.write(str(cont)+'. Journal:\n')
                index=[]
                keys=['journalVolume', 'journalIssue','journalPubDate']
                specified_keys=[]
                specified_keys = [element for element in keys if element in journal_keys]
                extracted_values = find_keys(journal_keys, specified_keys, journal_values)
                for entry in extracted_values:
                    for key, value in entry.items():
                        formatted_key = format_key(key)
                        f.write('\t'+f'{formatted_key}: {value}\n')
                    f.write('\n')
            if 'journalArticleType' in journal_keys:
                cont+=1
                f.write(str(cont)+'.  Type of Article:\n')
                f.write('\t'+journal_values[journal_keys.index('journalArticleType')]+'\n\n')
        f.write('FILE OVERVIEW\n----------------------\n')
        for i in range(0,len(filemetadata_keys)):
            f.write('\t'+'File name: '+filemetadata_values[i][filemetadata_keys[i].index('filename')]+'\n')
            if 'description' in filemetadata_keys[i]:
                f.write('\t'+'Decription: '+filemetadata_values[i][filemetadata_keys[i].index('description')]+'\n')
            f.write('\t'+'File format: '+filemetadata_values[i][filemetadata_keys[i].index('contentType')]+'\n\n')
        print('The Readme has been created in the directory' + path +'.')

# Checking if both inputs are provided
if not doi or not token:
    print("Please enter DOI, Token and URL of the repository correctly.")
else:
    api = NativeApi(base_url, token)
    data_api = DataAccessApi(base_url, token)
    dataset = api.get_dataset(doi)

    #  Metadata lists:
    citation_keys, geo_keys, social_keys, astronomy_keys, biomedical_keys, journal_keys = [[] for _ in range(6)]
    citation_values, geo_values, social_values, astronomy_values, biomedical_values, journal_values = [[] for _ in range(6)]
    filemetadata_keys=[]
    filemetadata_values=[]

    # Exporting metadata and creating readme
    exportmetadata(base_url, token, doi, citation_keys, citation_values, geo_keys, geo_values, social_keys,
                   social_values, astronomy_keys, astronomy_values, biomedical_keys, biomedical_values,
                   journal_keys, journal_values)
    filemetadata(base_url, token, doi, filemetadata_keys, filemetadata_values)
    createreadme(base_url, token, doi, citation_keys, citation_values, geo_keys, geo_values, social_keys,
                 social_values, astronomy_keys, astronomy_values, biomedical_keys, biomedical_values,
                 journal_keys, journal_values, filemetadata_keys, filemetadata_values)

    # Construct the correct file path
file_path = os.path.join(f'{doi.replace("doi:10.34810/", "")}', 'Readme.txt')

if IN_COLAB:
    # Running in Google Colab
    # Provide a download button
    download_button = widgets.Button(description="Download Readme")
    display(download_button)

    # Function to be executed when the download button is clicked
    def on_download_button_click(b):
        # Download the Readme.txt file in Google Colab
        files.download(file_path)

    # Event handler for the download button
    download_button.on_click(on_download_button_click)

else:
    # Running in Jupyter Notebook
    # Provide a download link
    download_link = FileLink(file_path, result_html_prefix="Click to view the Readme: ")
    display(download_link)